# Demo 01 - Using the toshi module

I/Q data reader is abstracted into a separate module named `toshi`. This notebook illustrates the usage of the module and calculation of reflectivity and radial velocity using the traditional pulse pair method.

The datasets were collected with phase coding, which is embeded in each pulse header, so it must be decoded prior to subsequent analysis.

In [ ]:
import os
import time
import numpy as np
import scipy.signal
import matplotlib
import matplotlib.pyplot as plt

import toshi

file = os.path.expanduser('~/Downloads/20200614_150015.256007-87-02.iqData.XXXX.AKITA.dat')
# file = os.path.expanduser('~/Downloads/20200615_010423.194421-8C-02.iqData.XXXX.AKITA.dat')
filesize = os.path.getsize(file)

plt.style.use('./darkmode.style')

In [ ]:
s = time.time()
ray_pulses, cpi_headers = toshi.read(file)
e = time.time()
print('Data read in {:.2f} s'.format(e - s))

In [ ]:
pulses = ray_pulses[0]

In [ ]:
ph = np.zeros(len(pulses), dtype=np.single)
for k, pulse in enumerate(pulses):
    ph[k] = pulse.phase_h_long

In [ ]:
fig, ax = plt.subplots(figsize=(8, 3), dpi=120)
plt.stem(-ph / np.pi, use_line_collection=True)
ax.set_ylim(-0.1, 1.1)
ax.set_xlabel('Samples')
ax.set_ylabel(r'Phase ($\times \pi$)')
ax.set_title('Transmit Phase')
plt.show()

In [ ]:
naz = len(ray_pulses)
ngate = ray_pulses[0][0].ngate_long_hi
# ngate = ray_pulses[0][0].ngate_short_hi
z = np.zeros((naz, ngate), dtype=np.single)
v = np.zeros((naz, ngate), dtype=np.single)

for k, (pulses, cpi_header) in enumerate(zip(ray_pulses, cpi_headers)):
    p = np.zeros((len(pulses), ngate), dtype=np.csingle)
    for j, pulse in enumerate(pulses):
        p[j, :] = pulse.h_long_hi * np.exp(-1j * pulse.phase_h_long)
    pp = p[1:, :] * np.conj(p[:-1, :])
    v[k, :] = np.angle(np.sum(pp, axis=0))
    z[k, :] = np.sum(np.abs(p), axis=0)

# Sweep operations
z = 20 * np.log10(z) - 60
# v[z < -10] = np.nan

In [ ]:
plt.figure(figsize=(9, 16), dpi=120)
plt.subplot(3, 1, 1)
# plt.imshow(z[:, :400].T, vmin=-15, vmax=45, interpolation='none')
plt.imshow(z[::2, :200].T, vmin=-15, vmax=45, interpolation='none')
plt.colorbar()
plt.title(os.path.basename(file))

cmap = matplotlib.cm.RdYlGn
vmin, vmax = -3, 3
# va1 = 0.05 * 1500 / 4
# va2 = 0.05 * 1800 / 4
va1 = 0.05 * 1800 / 4
va2 = 0.05 * 1500 / 4

v1 = va1 * v[::2, :]
v2 = va2 * v[1::2, :]
# vr = np.zeros((naz, ngate), dtype=np.single)
# vr[::2, :] = v1
# vr[::2, :] = v2

plt.subplot(3, 1, 2)
# plt.imshow(v1[:, :400].T, cmap=cmap, vmin=-np.pi, vmax=np.pi, interpolation='none')
# plt.imshow(v1[::2, :200].T, cmap=cmap, vmin=-np.pi, vmax=np.pi, interpolation='none')
plt.imshow(v1[:, :200].T, cmap=cmap, vmin=vmin, vmax=vmax, interpolation='none')
plt.colorbar()

plt.subplot(3, 1, 3)
# plt.imshow(v2[:, :400].T, cmap=cmap, vmin=-np.pi, vmax=np.pi, interpolation='none')
plt.imshow(v1[:, :200].T, cmap=cmap, vmin=vmin, vmax=vmax, interpolation='none')
plt.colorbar()

plt.show()

In [ ]:
a = 30
pulses = ray_pulses[a]
ngate = pulses[0].ngate_long_hi
p = np.zeros((len(pulses), ngate), dtype=np.csingle)
for j, pulse in enumerate(pulses):
    #print('AZ:{:.2f}'.format(pulse.azimuth))
    p[j, :] = pulse.h_long_hi * np.exp(-1j * pulse.phase_h_long)
# w = scipy.signal.get_window('hamming', len(p)).reshape((p.shape[0], 1))
# p = p * w

In [ ]:
# for a in range(20, 30):
#     pulses = ray_pulses[a]
#     s = np.zeros(len(pulses), dtype=np.csingle)
#     for k, pulse in enumerate(pulses):
#         s[k] = pulse.h_pilot_hi

#     fig = plt.figure(dpi=120)
#     plt.plot(np.real(s))
#     plt.plot(np.imag(s))    
# #     plt.plot(np.real(s[::2]))
# #     plt.plot(np.imag(s[::2]))

In [ ]:
plt.close()

In [ ]:
# g = 20
# w = scipy.signal.get_window('hamming', len(p)).reshape((p.shape[0], 1))
# np.fft.fft(p[:, g] * w, 512)

In [ ]:
# Windowed pulses
w = scipy.signal.get_window('hamming', len(p)).reshape((p.shape[0], 1))
wp = p * w

# for g in range(280, 285):
for g in range(20, 25):
    fig, axes = plt.subplots(nrows=2, ncols=1, dpi=120)
    axes[0].plot(np.real(p[:, g]))
    axes[0].plot(np.imag(p[:, g]))
    # axes[0].plot(np.real(p[::2, g]))
    # axes[0].plot(np.imag(p[::2, g]))
    axes[0].set_xlabel('Samples')
    axes[0].set_ylabel('Int-16')
    axes[0].set_title('Azimuth {} ({:.2f}-{:.2f} deg) Gate {}'.format(
        a, pulses[0].azimuth, pulses[-1].azimuth, g))
    
    s = np.fft.fftshift(np.fft.fft(wp[:, g], 512)) / len(p[:, g])
    f = np.arange(-256, 256) / 512 * np.pi
    axes[1].plot(f, 20 * np.log10(np.abs(s)), 'g')
    #axes[1].set_ylim((0, 10))
    axes[1].set_ylabel('Uncal. Power (dB)')
    axes[1].set_xlabel('Normalized Frequency (rad/sample)')
    
    fig.tight_layout(pad=0.8)

In [ ]:
0.05 * 1500 / 4